In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
doclist_txt[:5]

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

['corpus_csv\\Doyle_AStudyinScarlet.txt.csv',
 'corpus_csv\\Doyle_TheHoundoftheBaskervilles.txt.csv',
 'corpus_csv\\Doyle_TheSignoftheFour.txt.csv',
 'corpus_csv\\Howard_GodsoftheNorth.txt.csv',
 'corpus_csv\\Howard_SchadowsinZamboula.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[:1]

[['a',
  'scandal',
  'in',
  'bohemia',
  'a',
  'conan',
  'doyle',
  'i',
  'to',
  'sherlock',
  'holmes',
  'she',
  'is',
  'always',
  'the',
  'woman',
  'i',
  'have',
  'seldom',
  'heard',
  'him',
  'mention',
  'her',
  'under',
  'any',
  'other',
  'name',
  'in',
  'his',
  'eyes',
  'she',
  'eclipses',
  'and',
  'predominates',
  'the',
  'whole',
  'of',
  'her',
  'sex',
  'it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler',
  'all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind',
  'he',
  'was',
  'i',
  'take',
  'it',
  'the',
  'most',
  'perfect',
  'reasoning',
  'and',
  'observing',
  'machine',
  'that',
  'the',
  'world',
  'has',
  'seen',
  'but',
  'as',
  'a',
  'lover',
  'he',
  'would',
  'have',
  'placed',
  'himself',
  'in',
  'a',
  'false'

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow

0
doc_id token_id    
1      12292      2
       20485      1
       18433      2
       14594      1
       4107       1
       16397     11
       14         1
       12303      2
       8208       1
       19         1
       22         1
       3274       1
       14340      1
       20506      1
       27         1
       8220       1
       9562       1
       12319      1
       16417      1
       12322      7
       8227       1
       4133       1
       38         1
       4135       1
       4140       2
       10640      1
       23519      1
       4149       6
       8247      48
       8251       1
...              ..
15     4020       0
       18359      0
       24506      0
       8124       0
       18365      0
       6079       0
       18368      0
       20417      0
       24515      0
       10189      0
       8143       0
       14292      0
       15547      0
       6103       0
       20730      0
       4060       0
       8158       0
       2015       0
       18402      0
       12284      0
       6123       0
       4077       0
       2034       0
       10228      0
       20472      0
       14676      0
       12282      0
       4092       0
       12286      0
       23893      0

[56091 rows x 1 columns]

#### Convert to Doc-Topic Matrix

In [11]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [12]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))

with open(os.path.join(basepath, "Topics/tutorial_supplementals/stopwords/en"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [13]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [14]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

56091
50891


##### Create Feature Remove Lists

In [15]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

100

In [16]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

18472

##### Remove Features

In [17]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

In [18]:
len(clean_term_frequency)

8350

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [19]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

In [20]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [21]:
mm = MmCorpus("gb_plain.mm")

In [22]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [23]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [24]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [25]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [26]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [27]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id)

In [28]:
topic_nr_x = model.get_topic_terms(10)

[type2id[i[0]] for i in topic_nr_x]

['written',
 'hoar-frost',
 'sub-contractor',
 'rebounding',
 'real-hunts',
 'musty',
 'scour',
 'pitched',
 "farrier's",
 'bumblepuppy']

In [29]:
topics = model.show_topics(num_topics = 10)

In [30]:
topics

[(59,
  '0.045*"written" + 0.032*"rebounding" + 0.032*"hoar-frost" + 0.023*"sub-contractor" + 0.023*"real-hunts" + 0.017*"scour" + 0.014*"musty" + 0.013*"farrier\'s" + 0.013*"bumblepuppy" + 0.012*"pitched"'),
 (45,
  '0.044*"written" + 0.030*"rebounding" + 0.029*"hoar-frost" + 0.024*"sub-contractor" + 0.018*"musty" + 0.017*"real-hunts" + 0.013*"pitched" + 0.012*"farrier\'s" + 0.012*"bumblepuppy" + 0.012*"division"'),
 (31,
  '0.057*"written" + 0.031*"hoar-frost" + 0.028*"rebounding" + 0.024*"sub-contractor" + 0.019*"real-hunts" + 0.019*"musty" + 0.017*"farrier\'s" + 0.016*"scour" + 0.014*"bumblepuppy" + 0.012*"pitched"'),
 (4,
  '0.060*"written" + 0.031*"rebounding" + 0.028*"hoar-frost" + 0.018*"real-hunts" + 0.018*"sub-contractor" + 0.013*"scour" + 0.013*"farrier\'s" + 0.012*"division" + 0.011*"prisoners" + 0.011*"musty"'),
 (65,
  '0.036*"written" + 0.029*"hoar-frost" + 0.022*"rebounding" + 0.016*"sub-contractor" + 0.014*"real-hunts" + 0.014*"farrier\'s" + 0.012*"bumblepuppy" + 0.012

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [31]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))
basepath

'C:\\Users\\Philip\\Documents\\Python35-32_Scripts\\Funs\\DARIAH'

In [32]:
#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
#path_to_mallet = "mallet"
path_to_corpus = os.path.join(basepath, "Topics\\corpus_txt")

mal.create_mallet_binary(path_to_corpus)

#toDo absolute Pfade

['mallet', 'import-dir', '--input', 'C:\\Users\\Philip\\Documents\\Python35-32_Scripts\\Funs\\DARIAH\\Topics\\corpus_txt', '--output', 'mallet_output\\malletBinary.mallet', '--keep-sequence', '--remove-stopwords']


##### Mallet output erzeugen

In [33]:
outfolder = "mallet_output" 
path_to_mallet = "mallet"
malletBinary = "mallet_output\\malletBinary.mallet"
mal.create_mallet_model(malletBinary, outfolder)

## Visualisierung 

In [34]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

"lda_model = 'out_easy/corpus.lda'\ncorpus = 'out_easy/corpus.mm'\ndictionary = 'out_easy/corpus.dict'\ndoc_labels = 'out_easy/corpus_doclabels.txt'\ninteractive  = False\n\nvis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)"

#### Heatmap

In [35]:
#heatmap = visual.make_heatmap()

In [36]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [37]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [38]:
#vis.make_interactive()

In [39]:
#vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)